In [4]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn import preprocessing
import pandas_profiling as pdp
from utils import deal_price, deal_build_years, readConf, deal_size

conf, hostname = readConf()
s3_home = conf.get(hostname, "s3_home")
csv_path = s3_home + 'old_house.csv'
df = pd.read_csv(csv_path, dtype={'マンション名': 'str', '販売価格': 'str', '販売価格(万円)': 'float', '住所': 'str', '立地': 'str', '土地面積': 'str', '建物面積': 'str', '間取り': 'str', '築年月': 'str',
                                  '築年数': 'int', '詳細URL': 'str', '電話': 'str', '区域': 'str', '日付': 'str'}, encoding='utf-16', parse_dates=['日付'])
df['建物面積(m2)'] = df['建物面積'].apply(lambda x: deal_size(x))
df['土地面積(m2)'] = df['土地面積'].apply(lambda x: deal_size(x))

/Users/chankein/Documents/summoSpyder/config.ini


In [20]:
df_split1=df['立地'].str.split('「', expand=True)
df_split2=df_split1[1].str.split('」', expand=True)
df_split1=df_split1.loc[:,[0]]
df_split1.columns=['鉄道']
df_split2.columns=['駅','距離']
print(df_split1)
print(df_split2)

              鉄道
0      東京メトロ半蔵門線
1        ＪＲ総武線快速
2          ＪＲ山手線
3          ＪＲ山手線
4          ＪＲ山手線
...          ...
32653     ブルーライン
32654     ブルーライン
32655      ＪＲ横浜線
32656      ＪＲ横浜線
32657     ブルーライン

[32658 rows x 1 columns]
           駅     距離
0        神保町   徒歩1分
1        馬喰町   徒歩2分
2        秋葉原   徒歩6分
3        秋葉原   徒歩6分
4        秋葉原   徒歩6分
...      ...    ...
32653     中川   徒歩6分
32654  センター南  徒歩19分
32655     鴨居  徒歩20分
32656     鴨居  徒歩19分
32657  センター北  徒歩14分

[32658 rows x 2 columns]


In [19]:
df_location=pd.concat([df,df_split1,df_split2], axis=1)
print(df_location)

                                       マンション名      販売価格  販売価格(万円)  \
0      【神保町駅１分】１LDK＋EV　建物５２．９４㎡　日当り良好■整形地　南東…    6480万円    6480.0   
1                           東神田１（馬喰町駅） 9380万円    9380万円    9380.0   
2      ■山手線・日比谷線「秋葉原」駅徒歩6分、7沿線7駅以上利用可■北西角地■平…    9999万円    9999.0   
3                           東神田３（秋葉原駅） 9999万円    9999万円    9999.0   
4            築浅　清洲橋通り沿いの店舗事務所利便性の高い、7路線7駅利用可能    9999万円    9999.0   
...                                       ...       ...       ...   
32653                       中川２（中川駅） 1億6500万円  1億6500万円   16500.0   
32654                     平台（センター南駅） 1億7600万円  1億7600万円   17600.0   
32655                          池辺町（鴨居駅） 990万円     990万円     990.0   
32656                         池辺町（鴨居駅） 3080万円    3080万円    3080.0   
32657                      大棚町（センター北駅） 3680万円    3680万円    3680.0   

                  住所                  立地                   土地面積  \
0      東京都千代田区神田神保町１  東京メトロ半蔵門線「神保町」徒歩1分    36.62m2（11.07坪）（登記）   
1        東京都千代田区東神田１    ＪＲ総武線快速「馬喰町」徒

In [32]:
import re
def get_num(num_string):
    return int(re.findall(r"\d+",num_string)[0])

In [33]:
df_location['距離(分)']=df_location['距離'].apply(lambda x: get_num(x))

In [34]:
print(df_location['距離(分)'][:50])
print(df_location['距離'][:50])

0      1
1      2
2      6
3      6
4      6
5      6
6      6
7      6
8      6
9      3
10     3
11     4
12     7
13     3
14     2
15     3
16     6
17     2
18     3
19     1
20     4
21     3
22     4
23     3
24     3
25     4
26     2
27     2
28     3
29     3
30     3
31     3
32     2
33     3
34    11
35     7
36     8
37     6
38     6
39     6
40     9
41     9
42     2
43     7
44     8
45     6
46     8
47     4
48     6
49    11
Name: 距離(分), dtype: int64
0      徒歩1分
1      徒歩2分
2      徒歩6分
3      徒歩6分
4      徒歩6分
5      徒歩6分
6      徒歩6分
7      徒歩6分
8      徒歩6分
9      徒歩3分
10     徒歩3分
11     徒歩4分
12     徒歩7分
13     徒歩3分
14     徒歩2分
15     徒歩3分
16     徒歩6分
17     徒歩2分
18     徒歩3分
19     徒歩1分
20     徒歩4分
21     徒歩3分
22     徒歩4分
23     徒歩3分
24     徒歩3分
25     徒歩4分
26     徒歩2分
27     徒歩2分
28     徒歩3分
29     徒歩3分
30     徒歩3分
31     徒歩3分
32     徒歩2分
33     徒歩3分
34    徒歩11分
35     徒歩7分
36     徒歩8分
37     徒歩6分
38     徒歩6分
39     徒歩6分
40     徒歩9分
41     徒歩9分
42     徒歩2分
43     徒

747      2600万円※権利金含む200万円
2801            9998万8000円
3178            7322万5000円
4693     2380万円※権利金含む400万円
4985     1980万円※権利金含む200万円
8982     2600万円※権利金含む200万円
11006           9998万8000円
11392           7322万5000円
12921    2380万円※権利金含む400万円
13202    1980万円※権利金含む200万円
17203    2600万円※権利金含む200万円
19263           9998万8000円
19644           7322万5000円
21135    2380万円※権利金含む400万円
21414    1980万円※権利金含む200万円
Name: 販売価格, dtype: object
747      None
2801     None
3178     None
4693     None
4985     None
8982     None
11006    None
11392    None
12921    None
13202    None
17203    None
19263    None
19644    None
21135    None
21414    None
Name: 販売価格(万円), dtype: object


7322.5
